In [209]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [210]:
#load all ti matches
matches_df = pd.read_json('all_matches.json')
matches_df['match_id'].count()



100

In [211]:
# determine winner based on match_id
def get_winner(matchid):
    match_data = matches_df[matches_df['match_id']==matchid]
    radiant_win = match_data['radiant_win'].to_string(index=False)
    winner = 1
    if radiant_win == 'True':
        winner = 0
    return winner



In [212]:
#define heroes df
heroes_df = pd.read_json('heroes.json')
heroes_df = heroes_df[['id','localized_name']]
heroes_df.rename(columns={"localized_name":"hero_name"},inplace=True)
heroes_df.head()


,id,hero_name
0,1,Anti-Mage
1,2,Axe
2,3,Bane
3,4,Bloodseeker
4,5,Crystal Maiden


In [213]:
# define players df
ti_teams_df= pd.read_json('teams.json')
ti_teams = list(ti_teams_df['team_id'])
players_df = pd.read_json('proPlayers.json')
players_df = players_df[['account_id','personaname','name','team_id','team_name','team_tag']]
players_df = players_df[players_df['team_id'].isin(ti_teams)]
players_df.head()

,account_id,personaname,name,team_id,team_name,team_tag
17,10366616,Sneyking,Sneyking,8291895,Tundra Esports,Tundra
30,16497807,tOfu,tOfu,8599101,Gaimin Gladiators,GG
52,25907144,Let's Go,Cr1t-,39,Evil Geniuses,EG
83,38628747,Verissimus,MoonMeander,8260983,TSM FTX,TSM FTX
91,40547474,Aui_2000,Aui_2000,8291895,Tundra Esports,Tundra


In [214]:
#create pickbans df
picks_bans = matches_df.set_index('match_id').to_dict()['picks_bans']
picks = []
bans = []
for i in picks_bans:
    winner = get_winner(i)
    for j in picks_bans[i]:
        if j['is_pick']:
            pick = {}
            pick['match_id'] = i
            pick['order'] = j['order']
            pick['hero_id'] = j['hero_id']
            pick['team'] = j['team']
            if winner == j['team']:
                pick['win'] = 1
            else:
                pick['win'] = 0
            picks.append(pick)
        elif not j['is_pick']:
            ban = {}
            ban['match_id'] = i
            ban['order'] = j['order']
            ban['hero_id'] = j['hero_id']
            ban['team'] = j['team']
            bans.append(ban)

In [215]:
picks_df = pd.DataFrame(picks)
heroes_df['pick_count'] = heroes_df['id'].map(picks_df['hero_id'].value_counts())
wins = picks_df[['hero_id','win']].groupby(['hero_id']).sum()
wins = wins.to_dict()
heroes_df['wins'] = heroes_df['id'].map(wins['win'])
heroes_df['winrate'] = round(heroes_df['wins'] / heroes_df['pick_count'] * 100,2)

bans_df = pd.DataFrame(bans)
heroes_df['ban_count'] = heroes_df['id'].map(bans_df['hero_id'].value_counts())


heroes_df.head()

,id,hero_name,pick_count,wins,winrate,ban_count
0,1,Anti-Mage,1.0,1.0,100.0,1.0
1,2,Axe,2.0,0.0,0.0,2.0
2,3,Bane,4.0,2.0,50.0,1.0
3,4,Bloodseeker,2.0,0.0,0.0,32.0
4,5,Crystal Maiden,20.0,14.0,70.0,1.0


In [216]:
#create hero stats
players = matches_df.set_index('match_id').to_dict()['players']
hero_stats = []
for i in players:
    for j in players[i]:
        hero_stat = {}
        hero_stat['match_id'] = j['match_id']
        hero_stat['account_id'] = j['account_id']
        hero_stat['hero_id'] = j['hero_id']
        hero_stat['kills'] = j['kills']
        hero_stat['deaths'] = j['deaths']
        hero_stat['assists'] = j['assists']
        hero_stat['last_hits'] = j['last_hits']
        hero_stat['xp_per_min'] = j['xp_per_min']
        hero_stat['gold_per_min'] = j['gold_per_min']
        hero_stats.append(hero_stat)
hero_stats_df = pd.DataFrame(hero_stats)
players_teams = players_df.set_index('account_id').to_dict()['team_id']
hero_stats_df['team_id'] = hero_stats_df['account_id'].map(players_teams) 

hero_stats_df.head()

,match_id,account_id,hero_id,kills,deaths,assists,last_hits,xp_per_min,gold_per_min,team_id
0,6806663153,86799300,102,0,4,7,51,361,222,7390454
1,6806663153,221666230,52,6,4,5,300,578,526,7390454
2,6806663153,87063175,137,6,4,6,178,462,378,7390454
3,6806663153,108452107,10,2,0,1,331,647,535,7390454
4,6806663153,86726887,128,2,6,10,42,301,187,7390454


In [217]:
hero_avg_stats = hero_stats_df.groupby('hero_id').mean()
hero_max_stats = hero_stats_df.groupby('hero_id').max()
hero_min_stats = hero_stats_df.groupby('hero_id').min()

heroes_df['avg_kills'] = heroes_df['id'].map(hero_avg_stats['kills'])

heroes_df['max_kills'] = heroes_df['id'].map(hero_max_stats['kills'])

heroes_df['avg_deaths'] = heroes_df['id'].map(hero_avg_stats['deaths'])

heroes_df['min_deaths'] = heroes_df['id'].map(hero_min_stats['deaths'])

heroes_df['avg_assists'] = heroes_df['id'].map(hero_avg_stats['assists'])

heroes_df['max_assists'] = heroes_df['id'].map(hero_max_stats['assists'])

heroes_df['avg_last_hits'] = heroes_df['id'].map(hero_avg_stats['last_hits'])

heroes_df['max_last_hits'] = heroes_df['id'].map(hero_max_stats['last_hits'])

heroes_df['avg_xp_per_min'] = heroes_df['id'].map(hero_avg_stats['xp_per_min'])

heroes_df['avg_gold_per_min'] = heroes_df['id'].map(hero_avg_stats['gold_per_min'])

heroes_df.head()

,id,hero_name,pick_count,wins,winrate,ban_count,avg_kills,max_kills,avg_deaths,min_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min
0,1,Anti-Mage,1.0,1.0,100.0,1.0,7.00,7.0,6.00,6.0,13.00,13.0,471.00,471.0,927.0,700.00
1,2,Axe,2.0,0.0,0.0,2.0,5.00,8.0,5.00,4.0,3.00,6.0,205.50,229.0,535.5,437.00
2,3,Bane,4.0,2.0,50.0,1.0,1.00,2.0,4.75,2.0,10.75,17.0,27.75,41.0,364.5,236.75
3,4,Bloodseeker,2.0,0.0,0.0,32.0,9.00,14.0,4.00,4.0,10.00,12.0,396.50,466.0,698.0,601.50
4,5,Crystal Maiden,20.0,14.0,70.0,1.0,2.75,5.0,6.10,2.0,16.40,28.0,67.00,146.0,430.7,302.55


In [218]:
heroes_5_df = heroes_df[heroes_df['pick_count']>=5]

In [219]:
player_avg_stats = hero_stats_df.groupby('account_id').mean()
player_max_stats = hero_stats_df.groupby('account_id').max()
player_min_stats = hero_stats_df.groupby('account_id').min()

players_df['avg_kills'] = players_df['account_id'].map(player_avg_stats['kills'])

players_df['max_kills'] = players_df['account_id'].map(player_max_stats['kills'])

players_df['avg_deaths'] = players_df['account_id'].map(player_avg_stats['deaths'])

players_df['min_deaths'] = players_df['account_id'].map(player_min_stats['deaths'])

players_df['avg_assists'] = players_df['account_id'].map(player_avg_stats['assists'])

players_df['max_assists'] = players_df['account_id'].map(player_max_stats['assists'])

players_df['avg_last_hits'] = players_df['account_id'].map(player_avg_stats['last_hits'])

players_df['max_last_hits'] = players_df['account_id'].map(player_max_stats['last_hits'])

players_df['avg_xp_per_min'] = players_df['account_id'].map(player_avg_stats['xp_per_min'])

players_df['avg_gold_per_min'] = players_df['account_id'].map(player_avg_stats['gold_per_min'])

players_df['max_gold_per_min'] = players_df['account_id'].map(player_max_stats['gold_per_min'])

player_unique_heroes = hero_stats_df[['account_id','hero_id']].groupby('account_id').nunique()
players_df['unique_heroes'] = players_df['account_id'].map(player_unique_heroes['hero_id'])

players_df.head()

,account_id,personaname,name,team_id,team_name,team_tag,avg_kills,max_kills,avg_deaths,min_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min,max_gold_per_min,unique_heroes
17,10366616,Sneyking,Sneyking,8291895,Tundra Esports,Tundra,3.0,6.0,6.7,2.0,15.8,29.0,65.2,125.0,460.4,304.4,363.0,5.0
30,16497807,tOfu,tOfu,8599101,Gaimin Gladiators,GG,2.8,7.0,5.9,2.0,14.2,38.0,78.7,105.0,442.9,300.1,395.0,7.0
52,25907144,Let's Go,Cr1t-,39,Evil Geniuses,EG,4.8,12.0,4.6,0.0,15.6,23.0,49.9,82.0,468.1,309.7,376.0,7.0
83,38628747,Verissimus,MoonMeander,8260983,TSM FTX,TSM FTX,3.2,15.0,7.2,2.0,11.3,23.0,63.6,109.0,417.5,272.3,423.0,7.0
91,40547474,Aui_2000,Aui_2000,8291895,Tundra Esports,Tundra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
ti_teams_df = ti_teams_df[['team_id','name','tag']]
teams_stats_df = hero_stats_df.groupby(['match_id','team_id']).sum()
team_avg_stats_df = teams_stats_df.groupby('team_id').mean()
team_max_stats_df = teams_stats_df.groupby('team_id').max()
team_min_stats_df = teams_stats_df.groupby('team_id').min()

ti_teams_df['avg_kills'] = ti_teams_df['team_id'].map(team_avg_stats_df['kills'])
ti_teams_df['avg_deaths'] = ti_teams_df['team_id'].map(team_avg_stats_df['deaths'])
ti_teams_df['avg_assists'] = ti_teams_df['team_id'].map(team_avg_stats_df['assists'])
ti_teams_df['max_kills'] = ti_teams_df['team_id'].map(team_max_stats_df['kills'])
ti_teams_df['min_deaths'] = ti_teams_df['team_id'].map(team_min_stats_df['deaths'])
ti_teams_df['max_assists'] = ti_teams_df['team_id'].map(team_max_stats_df['assists'])

team_unique_heroes = hero_stats_df[['team_id','hero_id']].groupby('team_id').nunique()
ti_teams_df['unique_heroes'] = ti_teams_df['team_id'].map(team_unique_heroes['hero_id'])
ti_teams_df.head()

,team_id,name,tag,avg_kills,avg_deaths,avg_assists,max_kills,min_deaths,max_assists,unique_heroes
0,8597976,Talon,TLN,20.5,31.5,44.3,46,10,81,35
1,7732977,BOOM Esports,BOOM,25.1,23.6,59.3,52,6,132,29
2,7390454,Soniqs,SQ,21.0,23.5,45.8,40,9,99,33
3,2163,Team Liquid,Liquid,26.9,25.0,62.1,42,10,104,36
4,8260983,TSM FTX,TSM FTX,20.5,27.6,45.4,44,12,99,29


In [221]:
def get_matchwinner(row):
    if row['radiant_win'] == True:
        winner = 'radiant_team_id'
    else:
        winner = 'dire_team_id'
    return row[winner]

def convert(row,column):
    seconds = row[column]
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
     
    return "%d:%02d:%02d" % (hour, minutes, seconds)

ti_matches_df = matches_df[['match_id','duration','radiant_win','radiant_score','radiant_team_id','dire_score','dire_team_id']]
ti_matches_df['total_score'] = ti_matches_df['radiant_score']+ti_matches_df['dire_score']

ti_matches_df['team_winner'] = ti_matches_df.apply(lambda x: get_matchwinner(x), axis=1)
ti_matches_df['team_winner'] = ti_matches_df['team_winner'].map(ti_teams_df.set_index('team_id').to_dict()['name'])
ti_matches_df['duration_minutes'] = ti_matches_df.apply(lambda x: convert(x,'duration'), axis=1)
ti_matches_df.head()

,match_id,duration,radiant_win,radiant_score,radiant_team_id,dire_score,dire_team_id,total_score,team_winner,duration_minutes
0,6806663153,2178,False,16,7390454,18,15,34,PSG.LGD,0:36:18
1,6806047576,2971,False,23,7732977,24,2586976,47,OG,0:49:31
2,6806638898,3098,False,19,7732977,28,6209804,47,Royal Never Give Up,0:51:38
3,6806648571,2250,True,33,8131728,14,8599101,47,Hokori,0:37:30
4,6806662696,1455,False,5,2586976,34,39,39,Evil Geniuses,0:24:15


In [222]:
radiant_teams = ti_matches_df[['match_id','radiant_team_id','duration']].rename(columns={"radiant_team_id":"team_id"})
dire_teams = ti_matches_df[['match_id','dire_team_id','duration']].rename(columns={"dire_team_id":"team_id"})
ti_teams_matches = pd.concat([radiant_teams,dire_teams],ignore_index=True)
ti_teams_df['avg_length'] = ti_teams_df['team_id'].map(ti_teams_matches.groupby('team_id').mean()['duration'])
ti_teams_df['avg_length_minutes'] = ti_teams_df.apply(lambda x: convert(x,'avg_length'), axis=1)

In [223]:
hero_list = []
player_list = []
team_list = []

# Hero with the most picked:

In [224]:
hero_most_picked = heroes_df[heroes_df['pick_count']==heroes_df['pick_count'].max()][['hero_name','pick_count']].rename(columns={"hero_name":"answer","pick_count":"value"})
hero_list.append(hero_most_picked.to_dict('r'))
hero_most_picked

,answer,value
121,Marci,41.0


# Hero with the most banned:

In [225]:
hero_most_banned = heroes_df[heroes_df['ban_count']==heroes_df['ban_count'].max()][['hero_name','ban_count']]
hero_list.append(hero_most_banned.to_dict('r'))
hero_most_banned

,hero_name,ban_count
31,Enigma,66.0


# Hero with the highest win rate:

In [226]:
hero_max_winrate = heroes_5_df[heroes_5_df['winrate']==heroes_5_df['winrate'].max()][['hero_name','winrate']]
hero_list.append(hero_max_winrate.to_dict('r'))
hero_max_winrate

,hero_name,winrate
62,Jakiro,87.5


# Hero with the highest average kill:

In [227]:
hero_max_avgkills = heroes_5_df[heroes_5_df['avg_kills']==heroes_5_df['avg_kills'].max()][['hero_name','avg_kills']]
hero_list.append(hero_max_avgkills.to_dict('r'))
hero_max_avgkills


,hero_name,avg_kills
33,Sniper,9.777778


# Hero with the highest average assist:

In [228]:
heroes_5_df[heroes_5_df['avg_assists']==heroes_5_df['avg_assists'].max()][['hero_name','avg_assists']]

,hero_name,avg_assists
86,Nyx Assassin,17.157895


# Hero with the lowest average death:

In [229]:
heroes_5_df[heroes_5_df['avg_deaths']==heroes_5_df['avg_deaths'].min()][['hero_name','avg_deaths']]

,hero_name,avg_deaths
52,Lifestealer,2.0


# Hero with the highest average last hit:

In [230]:
heroes_5_df[heroes_5_df['avg_last_hits']==heroes_5_df['avg_last_hits'].max()][['hero_name','avg_last_hits']]

,hero_name,avg_last_hits
87,Naga Siren,545.583333


# Hero with the highest average xp per minute:

In [231]:
heroes_5_df[heroes_5_df['avg_xp_per_min']==heroes_5_df['avg_xp_per_min'].max()][['hero_name','avg_xp_per_min']]

,hero_name,avg_xp_per_min
44,Templar Assassin,773.0


# Hero with the most kills in a game:

In [232]:
heroes_5_df[heroes_5_df['max_kills']==heroes_5_df['max_kills'].max()][['hero_name','max_kills']]

,hero_name,max_kills
50,Leshrac,22.0


# Hero with the most last hits in a game:

In [233]:
heroes_5_df[heroes_5_df['max_last_hits']==heroes_5_df['max_last_hits'].max()][['hero_name','max_last_hits']]

,hero_name,max_last_hits
33,Sniper,1058.0


# Player with the highest kill average:

In [234]:
players_df[players_df['avg_kills']==players_df['avg_kills'].max()][['name','avg_kills']]

,name,avg_kills
723,Somnus,10.6


# Player with the highest assist average:

In [235]:
players_df[players_df['avg_assists']==players_df['avg_assists'].max()][['name','avg_assists']]

,name,avg_assists
1267,kaka,17.4


# Player with the lowest death average:

In [236]:
players_df[players_df['avg_deaths']==players_df['avg_deaths'].min()][['name','avg_deaths']]

,name,avg_deaths
1056,Nightfall 凛,1.2


# Player with the highest last hit average:

In [237]:
players_df[players_df['avg_last_hits']==players_df['avg_last_hits'].max()][['name','avg_last_hits']]

,name,avg_last_hits
2454,JACKBOYS,524.8


# Player with the highest GPM average:

In [238]:
players_df[players_df['avg_gold_per_min']==players_df['avg_gold_per_min'].max()][['name','avg_gold_per_min']]

,name,avg_gold_per_min
2236,YATOROGOD,724.3


# Player with the most kills:

In [239]:
players_df[players_df['max_kills']==players_df['max_kills'].max()][['name','max_kills']]

,name,max_kills
723,Somnus,22.0


# Player with the most assists:

In [240]:
players_df[players_df['max_assists']==players_df['max_assists'].max()][['name','max_assists']]

,name,max_assists
1418,TIMS,39.0


# Player with the most last hits:

In [241]:
players_df[players_df['max_last_hits']==players_df['max_last_hits'].max()][['name','max_last_hits']]

,name,max_last_hits
2454,JACKBOYS,1058.0


# Player with the most GPM:

In [242]:
players_df[players_df['max_gold_per_min']==players_df['max_gold_per_min'].max()][['name','max_gold_per_min']]

,name,max_gold_per_min
909,dyrachyo,995.0


# Player with the most different heroes:

In [243]:
players_df[players_df['unique_heroes']==players_df['unique_heroes'].max()][['name','unique_heroes']]

,name,unique_heroes
2669,萧瑟,10.0


# Team with the most kills in a game:

In [244]:
ti_teams_df[ti_teams_df['max_kills']==ti_teams_df['max_kills'].max()][['name','max_kills']]

,name,max_kills
7,Gaimin Gladiators,54


# Team with the highest kill average:

In [245]:
ti_teams_df[ti_teams_df['avg_kills']==ti_teams_df['avg_kills'].max()][['name','avg_kills']]

,name,avg_kills
13,Evil Geniuses,30.1


# Team with the least deaths in a game:

In [246]:
ti_teams_df[ti_teams_df['min_deaths']==ti_teams_df['min_deaths'].min()][['name','min_deaths']]

,name,min_deaths
15,Team Aster,3


# Team with the most assists in a game:

In [247]:
team_most_assists = ti_teams_df[ti_teams_df['max_assists']==ti_teams_df['max_assists'].max()][['name','max_assists']]
team_most_assists

,name,max_assists
1,BOOM Esports,132
7,Gaimin Gladiators,132
14,Fnatic,132


# Team who wins the longest game:

In [248]:
ti_matches_df[ti_matches_df['duration']==ti_matches_df['duration'].max()][['team_winner','duration_minutes']]

,team_winner,duration_minutes
49,Royal Never Give Up,1:10:29


# Team who wins the shortest game:

In [249]:
ti_matches_df[ti_matches_df['duration']==ti_matches_df['duration'].min()][['team_winner','duration_minutes']]



,team_winner,duration_minutes
14,Team Secret,0:19:30


# Team with the highest length average:

In [250]:
ti_teams_df[ti_teams_df['avg_length']==ti_teams_df['avg_length'].max()][['name','avg_length_minutes']]

,name,avg_length_minutes
1,BOOM Esports,0:47:19


# Team with the most different heroes:

In [251]:
ti_teams_df[ti_teams_df['unique_heroes']==ti_teams_df['unique_heroes'].max()][['name','unique_heroes']]

,name,unique_heroes
3,Team Liquid,36
9,beastcoast,36


# Team with the fewest different heroes:

In [252]:
ti_teams_df[ti_teams_df['unique_heroes']==ti_teams_df['unique_heroes'].min()][['name','unique_heroes']]

,name,unique_heroes
11,Entity,24
